# SimResults To Race Reports and Championship Tables

In [1]:
import pandas as pd
import numpy as np
import re, os

### Race Result Reader

In [2]:
class RaceReport():
    
    # Race report table format strings
    qualy_row_0 = """(% border="1" style="width:554px" %)"""
    qualy_row_1 = """|=(% scope="row" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 41px; text-align: center;" %)Pos|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)No|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 155px; text-align: center;" %)Driver|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 233px; text-align: center;" %)Team|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 77px; text-align: center;" %)Time"""
    qualy_row_2 = """|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="border-color:#000000; text-align:center; width:44px" %){}|(% style="border-color:#000000; width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; border-color:#000000; width:233px" %){}|(% style="text-align:center; border-color:#000000; width:77px" %){}"""
    qualy_row = """|=(% style="background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="text-align:center; width:44px" %){}|(% style="width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; width:233px" %){}|(% style="text-align:center; width:77px" %){}"""
    qualy_row_last = """|=(% colspan="5" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 552px;" %)[[Source>>{}]]"""
    race_row_0 = """(% border="1" style="width:747px" %)"""
    race_row_1 = """|=(% scope="row" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 41px; text-align: center;" %)Pos|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)No|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 155px; text-align: center;" %)Driver|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 229px; text-align: center;" %)Team|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)Laps|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 111px; text-align: center;" %)Time/Retired|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 53px; text-align: center;" %)Grid|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 62px; text-align: center;" %)Points"""
    race_row_2 = """|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="border-color:#000000; text-align:center; width:44px" %){}|(% style="border-color:#000000; width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; border-color:#000000; width:233px" %){}|(% style="text-align:center; border-color:#000000; width:44px" %){}|(% style="text-align:center; border-color:#000000; width:111px" %){}|(% style="text-align:center; border-color:#000000; width:53px" %){}|(% style="text-align:center; border-color:#000000; width:62px" %){}"""
    race_row = """|=(% style="background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="text-align:center; width:44px" %){}|(% style="width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; width:233px" %){}|(% style="text-align:center; width:44px" %){}|(% style="text-align:center; width:111px" %){}|(% style="text-align:center; width:53px" %){}|(% style="text-align:center; width:62px" %){}"""
    race_row_last = """|=(% colspan="8" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 745px;" %)[[Source>>{}]]"""
    race_row_fastestlap = """Fastest lap:  [[image:{}||height="14" width="23"]] {} - {}"""
    
    
    # Read series drivers info
    def read_drivers_table(series_directory):
        drivers_table_file = "{}/drivers_table.csv".format(series_directory)
        return pd.read_csv(drivers_table_file, dtype=object)
    
    
    # Read series points scoring info
    def read_points_table(series_directory):
        points_table_file = "{}/points_table.csv".format(series_directory)
        table = pd.read_csv(points_table_file, dtype=object)
        table = table.astype(int)
        return table
    
    
    # Read series track info (used only for championship table)
    def read_tracks_table(series_directory):
        tracks_table_file = "{}/tracks_table.csv".format(series_directory)
        table = pd.read_csv(tracks_table_file, dtype=object)
        table["csv_manual_adjustment"] = table["csv_manual_adjustment"].astype(int)
        return table
    
    
    # Constructor, optionally pass in already-parsed drivers and points table info, debug to print intermediate dataframes
    def __init__(self, table_names, series_directory, race_directory, drivers_table=None, points_table=None, csv_manual_adjustment=0, output_file_name="wiki_tables.txt", debug=False):
        
        self.table_names = table_names
        self.debug = debug
        
        if self.debug: print("Creating race report:", series_directory, race_directory)
        
        self.drivers_table = drivers_table if drivers_table is not None else RaceReport.read_drivers_table(series_directory)
        self.points_table = points_table if points_table is not None else RaceReport.read_points_table(series_directory)

        self.output_file = "{}/{}/{}".format(series_directory, race_directory, output_file_name)
        
        simresults_file_name = [file for file in os.listdir("{}/{}".format(series_directory, race_directory)) if file.endswith(".csv")][0]
        simresults_code = os.path.splitext(simresults_file_name)[0]
        self.results_file = "{}/{}/{}.csv".format(series_directory, race_directory, simresults_code)
        self.simresults_url = "https://simresults.net/{}".format(simresults_code)
        
        self.csv_manual_adjustment = csv_manual_adjustment
        self.tables = self.__read_results_tables()
        self.__clean_results_tables()
    
    
    # Return race result pandas dataframes in dict keyed by table_names
    def __read_results_tables(self):
        
        rows = {name: [0,0] for name in self.table_names}
        with open(self.results_file) as fp:
            current_table = ""
            for i, row in enumerate(fp):
                if current_table:
                    if row=="\n":
                        rows[current_table][1] = i
                        if self.debug: print("ending line {}".format(rows[current_table][1]))
                        current_table = ""
                else:
                    for name in self.table_names:
                        if row.startswith(name):
                            current_table = name
                            rows[current_table][0] = i+2
                            if self.debug: print("found table '{}' starting line {} ".format(current_table, rows[current_table][0]), end="")
        
        tables = {}
        for name in self.table_names:
            table_skiprows = rows[name][0] + self.csv_manual_adjustment
            table_nrows = rows[name][1] - table_skiprows - 1 + self.csv_manual_adjustment
            table_df = pd.read_csv(self.results_file, skiprows=table_skiprows, nrows=table_nrows, index_col=False, dtype=object)
            if self.debug: display(table_df)
            tables[name] = table_df
        
        return tables
    
    
    # Clean race result dataframes, cast numerical columns to integers, join points information and starting positions to driver rows
    def __clean_results_tables(self):
        
        # Merge starting position info to grid column of a race table from either quali session or previous race, optionally adding quali points column
        def merge_qualy_info(race_table, qualy_table, add_qualy_points=False):
            if add_qualy_points:
                qualy_table = qualy_table[["Driver", "Pos", "Points"]]
                qualy_table = qualy_table.rename(columns={"Pos": "Grid", "Points": "Qualify Points"})
            else:
                qualy_table = qualy_table[["Driver", "Pos"]]
                qualy_table = qualy_table.rename(columns={"Pos": "Grid"})

            race_table = race_table.merge(qualy_table, how='left', on="Driver")

            race_table["Grid"] = race_table["Grid"].fillna(-1)
            # Pandas issue where NaN values cause ints to become floats?
            race_table["Grid"] = race_table["Grid"].astype(int)
            race_table["Pos"] = race_table["Pos"].astype(int)
            race_table["Laps"] = race_table["Laps"].astype(int)

            if add_qualy_points:
                race_table["Qualify Points"] = race_table["Qualify Points"].fillna(0)
                race_table["Qualify Points"] = race_table["Qualify Points"].astype(int)

            return race_table
        
        # Strip quotes and whitespace from strings, cast position and laps to integers, drop unnamed columns and rows for non-participants
        for name, table_df in self.tables.items():

            table_df = table_df.apply(lambda s: s.str.strip(' \'"'), axis=1)
            table_df = table_df.rename(columns=lambda c: c.strip(' \'"'))
            table_df = table_df.apply(lambda s: s.str.replace('(.*\d\d*\.\d{3})0$', r'\1'), axis=1)

            table_df["Pos"] = table_df["Pos"].astype(int)
            table_df["Laps"] = table_df["Laps"].astype(int)

            table_df = table_df[table_df["Laps"] > 0]

            table_df = table_df[table_df.columns.drop(list(table_df.filter(regex='Unnamed*')))]
            table_df = table_df.dropna(how='all', axis='columns')

            self.tables[name] = table_df
            
            
        # Merge points column to both quali and race sessions, for race sessions: get attatched qualifying or previous race to get starting positions
        for name, table_df in self.tables.items():

            if name.startswith("Qualify"):

                points_column = pd.Series(np.zeros(len(table_df))).add(self.points_table["qualy_points"].astype(int), fill_value=0)
                table_df["Points"] = points_column.astype(int)

            if name.startswith("Race"):

                table_df["Time/Retired"] = table_df["Time/Retired"].str.replace('^\s*$', "")
                table_df["Consistency"] = table_df["Consistency"].str.replace('^-$', "")

                points_column = pd.Series(np.zeros(len(table_df))).add(self.points_table["points"].astype(int), fill_value=0)
                table_df["Points"] = points_column.astype(int)

                session_num = int(re.findall('(?:Qualify|Race)\s(\d+)\sresult|$', name)[0])
                qualy_table_name = "Qualify result" if session_num==1 and "Qualify result" in self.tables.keys() else "Qualify {} result".format(session_num)
                previous_race_table_name = "Race {} result".format(session_num - 1)
                if qualy_table_name in self.tables:
                    table_df = merge_qualy_info(table_df, self.tables[qualy_table_name], add_qualy_points=True)
                elif previous_race_table_name in self.tables:
                    table_df = merge_qualy_info(table_df, self.tables[previous_race_table_name])
                else:
                    table_df["Grid"] = ""    

            if self.debug: display(table_df)
            self.tables[name] = table_df
        

    # Generate quali table markdown from quali session dataframe, result is a list of lines
    def __generate_qualy_table_strings(self, table_name):

        table_df = self.tables[table_name]

        lines_buffer = [table_name, self.qualy_row_0, self.qualy_row_1]

        for i, df_row in table_df.iterrows():

            driver = df_row["Driver"]

            driver_info = self.drivers_table.loc[self.drivers_table["ign"]==driver]
            assert len(driver_info)==1, "Error looking up {} in drivers_table".format(driver)

            position = df_row["Pos"]
            number = driver_info["number"].item()
            flag = driver_info["flag"].item()
            name = driver_info["name"].item()
            team = driver_info["team"].item()
            time = df_row["Best lap"]
            if position==1:
                time = "**{}**".format(time)

            line = self.qualy_row_2.format(position, number, flag, name, team, time) if i==0 else self.qualy_row.format(position, number, flag, name, team, time)
            lines_buffer.append(line)

        lines_buffer.append(self.qualy_row_last.format(self.simresults_url))

        if self.debug: print("made {} rows for table: {}".format(len(lines_buffer), table_name))
        return lines_buffer
    
    
    def __generate_race_table_strings(self, table_name):
    
        table_df = self.tables[table_name]

        lines_buffer = [table_name, self.race_row_0, self.race_row_1]

        for i, df_row in table_df.iterrows():

            driver = df_row["Driver"]

            driver_info = self.drivers_table.loc[self.drivers_table["ign"]==driver]
            assert len(driver_info)==1, "Error looking up {} in drivers_table".format(driver)

            position = df_row["Pos"]
            number = driver_info["number"].item()
            flag = driver_info["flag"].item()
            name = driver_info["name"].item()
            team = driver_info["team"].item()
            laps = df_row["Laps"]
            points = df_row["Points"]
            timeorretired = df_row["Time/Retired"] if df_row["Time/Retired"] else "DNF"
            grid = df_row["Grid"] if df_row["Grid"]>0 else "DNQ"

            if "Qualify Points" in table_df.columns:
                points = str(df_row["Points"] + df_row["Qualify Points"]) + ("^^{}^^".format(grid) if df_row["Qualify Points"]>0 else "")

            line = self.race_row_2.format(position, number, flag, name, team, laps, timeorretired, grid, points) if i==0 else self.race_row.format(position, number, flag, name, team, laps, timeorretired, grid, points)
            lines_buffer.append(line)

        lines_buffer.append(self.race_row_last.format(self.simresults_url))

        # Find fastest lap in race results
        table_df_withtimes = table_df.copy()
        table_df_withtimes["Best lap"] = pd.to_datetime(table_df_withtimes["Best lap"], format="%M:%S.%f", errors='coerce')
        
        fastest_driver = table_df.loc[table_df_withtimes["Best lap"]==table_df_withtimes["Best lap"].min()].iloc[0]["Driver"]
        fastest_driver_flag = self.drivers_table.loc[self.drivers_table["ign"]==fastest_driver].iloc[0]["flag"]
        
        fastest_time = table_df.loc[table_df["Driver"]==fastest_driver]["Best lap"].item()
        lines_buffer.append(self.race_row_fastestlap.format(fastest_driver_flag, fastest_driver, fastest_time))

        if self.debug: print("made {} rows for table: {}".format(len(lines_buffer), table_name))
        return lines_buffer
    
    
    # Generate table markdown for all table_names, result is map from table names to table markdown string
    def generate_tables_strings(self):
        tables_strings = {}

        for name in self.table_names:
            lines_buffer = ["error!"]
            if name.startswith("Qualify"):
                lines_buffer = self.__generate_qualy_table_strings(name)
            elif name.startswith("Race"):
                lines_buffer = self.__generate_race_table_strings(name)
            tables_strings[name] = "\n".join(lines_buffer) + "\n\n"
            
        return tables_strings
    
    
    # Write table markdown for all table_names, optionally provide the generated table strings
    def write_generated_tables(self, tables_strings=None):
        if not tables_strings:
            tables_strings = self.generate_tables_strings()
            
        with open(self.output_file, "w+") as fp:
            for name in self.table_names:
                fp.write(tables_strings[name])
                if self.debug: print("wrote table {} to {}".format(name, self.output_file))

    

### Read Series Info

In [3]:
series = "PWC 2020"
series_sessions = ["Qualify result", "Race 1 result", "Race 2 result"]

series_drivers_table = RaceReport.read_drivers_table(series)
display(series_drivers_table)

series_points_table = RaceReport.read_points_table(series)
display(series_points_table)

series_tracks_table = RaceReport.read_tracks_table(series)
display(series_tracks_table)

,ign,name,number,team,flag
0,McLarenTim,McLarenTim,71,Timberwolves Racing,https://upload.wikimedia.org/wikipedia/en/thum...
1,PhotonBurst,PhotonBurst,28,Timberwolves Racing,https://upload.wikimedia.org/wikipedia/commons...
2,gunciz,Gunciz,000,Phoenix Racing Team Gold,https://upload.wikimedia.org/wikipedia/commons...
3,MrGrinder,MrGrinder,20,Phoenix Racing Team Gold,https://upload.wikimedia.org/wikipedia/en/thum...
4,Trempale,Trempale,17,Monterey SimSport,https://upload.wikimedia.org/wikipedia/en/thum...
5,JEMknight657,JEMknight657,15,Independent,https://upload.wikimedia.org/wikipedia/en/thum...
6,Waffles,Waffles,30,Independent,https://upload.wikimedia.org/wikipedia/en/thum...
7,Salzeder,Salzeder,16,DSW Racing,https://upload.wikimedia.org/wikipedia/en/thum...
8,NotJames,NotJames,73,Monterey SimSport,https://upload.wikimedia.org/wikipedia/en/thum...
9,xPsychedelix,xPsychedelix,14,Phoenix Racing Team Black,https://upload.wikimedia.org/wikipedia/en/thum...


,pos,points,qualy_points
0,1,40,3
1,2,36,2
2,3,32,1
3,4,28,0
4,5,25,0
5,6,22,0
6,7,19,0
7,8,16,0
8,9,13,0
9,10,10,0


,directory,abbrev,full_name,flag,csv_manual_adjustment
0,zolder,ZOL,Zolder,https://upload.wikimedia.org/wikipedia/commons...,-1
1,kyalami,KYA,Kyalami,https://upload.wikimedia.org/wikipedia/commons...,-1
2,cgv,VIL,Villeneuve,https://upload.wikimedia.org/wikipedia/commons...,-1
3,anderstorp,AND,Anderstorp,https://upload.wikimedia.org/wikipedia/en/thum...,-1
4,monza,MON,Monza,https://upload.wikimedia.org/wikipedia/en/thum...,-1
5,brno,BRN,Brno,https://upload.wikimedia.org/wikipedia/commons...,0
6,brands,BHA,Brands Hatch,https://upload.wikimedia.org/wikipedia/en/thum...,0
7,assen,ASS,Assen,https://upload.wikimedia.org/wikipedia/commons...,0
8,nurb,NUR,Nurburgring,https://upload.wikimedia.org/wikipedia/en/thum...,0
9,hock,HOC,Hockenheimring,https://upload.wikimedia.org/wikipedia/en/thum...,0


### Read Race Results

In [4]:
race_reports = {}

for i in series_tracks_table.index:
    race, adjustment = series_tracks_table["directory"][i], series_tracks_table["csv_manual_adjustment"][i]
    race_reports[race] = RaceReport(series_sessions, series, race, drivers_table=series_drivers_table, points_table=series_points_table, csv_manual_adjustment=adjustment)
    print(race)
    [display(race_reports[race].tables[session]) for session in series_sessions if session.startswith("Race")]

zolder


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Points,Grid,Qualify Points
0,1,,Timberwolves Racing,urd_t5_maures_2018,McLarenTim,16,22:45.433,01:24.273,99.32%,15,0,40,1,3
1,2,,Timberwolves Racing,urd_t5_maures_2018,PhotonBurst,16,+00:18.490,01:24.917,98.69%,0,0,36,3,1
2,3,,PRT Gold,urd_t5_aura_2018,MrGrinder,16,+00:23.170,01:24.941,98.49%,0,0,32,5,0
3,4,,PRT Gold,urd_t5_aura_2018,gunciz,16,+00:31.415,01:25.042,97.86%,0,0,28,4,0
4,5,,,urd_t5_aura_2018,Waffles,16,+00:44.662,01:26.230,98.62%,0,0,25,6,0
5,6,,,urd_t5_bayro_2018,JEMknight657,16,+00:48.254,01:26.094,98.64%,0,0,22,2,2
6,7,,M Power,urd_t5_bayro_2018,SBD,16,+00:51.262,01:26.023,97.90%,0,0,19,7,0
7,8,,PRT Black,urd_t5_aura_2018,gman197002,16,+01:01.628,01:26.978,98.22%,0,0,16,12,0
8,9,,,urd_t5_bayro_2018,NihonTiger,16,+01:15.478,01:27.660,98.31%,0,0,13,11,0
9,10,,PRT Black,urd_t5_aura_2018,Viperion_NZ,16,+01:25.947,01:27.462,96.91%,0,0,10,13,0


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Points,Grid
0,1,,Timberwolves Racing,urd_t5_maures_2018,McLarenTim,29,41:37.353,01:24.279,99.15%,26,0,40,1
1,2,,Timberwolves Racing,urd_t5_maures_2018,PhotonBurst,29,+00:16.476,01:24.339,98.60%,2,0,36,2
2,3,,PRT Gold,urd_t5_aura_2018,MrGrinder,29,+00:21.628,01:24.684,98.84%,0,0,32,3
3,4,,PRT Gold,urd_t5_aura_2018,gunciz,29,+00:25.517,01:24.834,98.93%,0,0,28,4
4,5,,,urd_t5_bayro_2018,JEMknight657,29,+00:54.525,01:25.302,98.23%,0,0,25,6
5,6,,M Power,urd_t5_bayro_2018,SBD,29,+00:57.661,01:25.581,98.73%,0,0,22,7
6,7,,,urd_t5_aura_2018,Waffles,29,+01:20.885,01:25.954,97.87%,0,0,19,5
7,8,,M Power,urd_t5_bayro_2018,bacsi97,29,+01:22.777,01:26.154,98.11%,0,0,16,11
8,9,,PRT Black,urd_t5_aura_2018,gman197002,29,+01:23.901,01:26.068,97.97%,0,0,13,8
9,10,,,urd_t5_bayro_2018,NihonTiger,28,+01:41.477,01:27.328,98.79%,0,0,10,9


kyalami


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Points,Grid,Qualify Points
0,1,,,urd_t5_maures_2018,McLarenTim,14,22:07.556,01:33.545,99.67%,9,0,40,2,2
1,2,,,urd_t5_maures_2018,PhotonBurst,14,+00:02.199,01:33.594,99.04%,4,0,36,1,3
2,3,,,urd_t5_aura_2018,MrGrinder,14,+00:05.578,01:33.807,99.54%,0,0,32,3,1
3,4,,,urd_t5_aura_2018,gunciz,14,+00:10.145,01:34.005,99.47%,0,0,28,4,0
4,5,,,urd_t5_aura_2018,Waffles,14,+00:16.178,01:34.520,99.25%,0,0,25,5,0
5,6,,,urd_t5_bayro_2018,bacsi97,14,+00:16.721,01:34.700,99.48%,0,0,22,7,0
6,7,,,urd_t5_bayro_2018,SBD,14,+00:33.477,01:34.294,98.12%,0,0,19,8,0
7,8,,,urd_t5_bayro_2018,NihonTiger,14,+00:39.760,01:36.190,99.28%,0,0,16,9,0
8,9,,,urd_t5_bayro_2018,NotJames,14,+00:41.120,01:35.539,98.97%,0,0,13,12,0
9,10,,,urd_t5_aura_2018,Viperion_NZ,14,+00:45.860,01:35.439,97.62%,0,0,10,6,0


...

,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Points,Grid,Qualify Points
0,1,,,urd_t5_maures_2018,McLarenTim,16,21:54.406,01:21.157,99.54%,13,0,40,2,2
1,2,,,urd_t5_maures_2018,PhotonBurst,16,+00:00.807,01:20.979,99.16%,2,0,36,1,3
2,3,,,urd_t5_aura_2018,MrGrinder,16,+00:03.672,01:21.057,99.14%,0,0,32,4,0
3,4,,,urd_t5_aura_2018,gunciz,16,+00:09.666,01:21.570,99.35%,0,0,28,3,1
4,5,,,urd_t5_aura_2018,Waffles,16,+00:11.325,01:21.240,98.81%,0,0,25,6,0
5,6,,,urd_t5_bayro_2018,JEMknight657,16,+00:11.693,01:21.644,99.35%,0,0,22,5,0
6,7,,,urd_t5_bayro_2018,NotJames,16,+00:32.565,01:22.597,98.89%,0,0,19,9,0
7,8,,,urd_t5_aura_2018,xPsychedelix,16,+00:32.839,01:22.355,98.70%,0,0,16,7,0
8,9,,,urd_t5_aura_2018,gman197002,16,+00:34.636,01:22.302,98.40%,0,0,13,8,0
9,10,,,urd_t5_bayro_2018,Trempale,16,+00:39.183,01:22.141,97.81%,0,0,10,12,0


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Points,Grid
0,1,,,urd_t5_maures_2018,McLarenTim,30,41:24.378,01:21.288,99.47%,25,0,40,1
1,2,,,urd_t5_maures_2018,PhotonBurst,30,+00:04.708,01:21.149,99.13%,4,0,36,2
2,3,,,urd_t5_bayro_2018,JEMknight657,30,+00:11.107,01:21.436,99.29%,0,0,32,6
3,4,,,urd_t5_aura_2018,MrGrinder,30,+00:11.636,01:21.375,99.15%,0,0,28,3
4,5,,,urd_t5_aura_2018,gunciz,30,+00:12.662,01:21.224,99.05%,0,0,25,4
5,6,,,urd_t5_aura_2018,Waffles,30,+00:26.831,01:21.708,98.91%,0,0,22,5
6,7,,,urd_t5_aura_2018,xPsychedelix,30,+00:49.221,01:22.036,98.63%,0,0,19,8
7,8,,,urd_t5_bayro_2018,NotJames,30,+00:53.287,01:22.024,98.52%,0,0,16,7
8,9,,,urd_t5_aura_2018,gman197002,30,+00:54.089,01:22.180,98.37%,0,0,13,9
9,10,,,urd_t5_bayro_2018,Trempale,30,+01:07.939,01:22.498,98.28%,0,0,10,10


hock


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Points,Grid,Qualify Points
0,1,,,urd_t5_maures_2018,McLarenTim,15,22:28.346,01:29.169,99.77%,14,0,40,1,3
1,2,,,urd_t5_maures_2018,PhotonBurst,15,+00:01.347,01:29.111,99.70%,0,0,36,2,2
2,3,,,urd_t5_aura_2018,gunciz,15,+00:18.071,01:30.014,99.60%,0,0,32,3,1
3,4,,,urd_t5_aura_2018,MrGrinder,15,+00:18.866,01:29.700,99.24%,0,0,28,4,0
4,5,,,urd_t5_bayro_2018,JEMknight657,15,+00:19.482,01:29.663,99.35%,0,0,25,6,0
5,6,,,urd_t5_aura_2018,Waffles,15,+00:28.873,01:30.027,98.95%,0,0,22,7,0
6,7,,,urd_t5_aura_2018,gman197002,15,+00:37.582,01:30.864,99.40%,0,0,19,11,0
7,8,,,urd_t5_bayro_2018,Salzeder,15,+00:38.589,01:30.552,99.01%,0,0,16,8,0
8,9,,,urd_t5_bayro_2018,Trempale,15,+00:44.545,01:29.873,97.31%,0,0,13,5,0
9,10,,,urd_t5_aura_2018,xPsychedelix,15,+01:00.145,01:31.127,97.91%,0,0,10,10,0


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Points,Grid
0,1,,,urd_t5_maures_2018,McLarenTim,28,42:14.763,01:29.104,98.64%,23,0,40,1
1,2,,,urd_t5_maures_2018,PhotonBurst,28,+00:28.398,01:29.157,98.72%,4,0,36,2
2,3,,,urd_t5_aura_2018,MrGrinder,28,+00:31.887,01:29.675,98.04%,0,0,32,4
3,4,,,urd_t5_bayro_2018,Trempale,28,+00:41.427,01:30.159,98.46%,0,0,28,9
4,5,,,urd_t5_bayro_2018,JEMknight657,28,+00:41.871,01:29.822,98.20%,0,0,25,5
5,6,,,urd_t5_aura_2018,Waffles,28,+00:50.833,01:30.462,98.27%,0,0,22,6
6,7,,,urd_t5_aura_2018,gman197002,28,+01:09.598,01:30.906,98.19%,0,0,19,7
7,8,,,urd_t5_bayro_2018,Salzeder,28,+01:10.441,01:30.775,97.99%,0,0,16,8
8,9,,,urd_t5_bayro_2018,NotJames,28,+01:11.550,01:30.846,98.36%,0,0,13,11
9,10,,,urd_t5_aura_2018,xPsychedelix,28,+01:13.040,01:30.253,97.97%,0,0,10,10


### Debug

In [5]:
# test_series_directory = "PWC 2020"
# test_race_directory = "hock"
# test_table_names = ["Qualify result", "Race 1 result", "Race 2 result"]

# testReport = RaceReport(test_table_names, test_series_directory, test_race_directory, debug=True)
# # testReport.write_generated_tables()